[![logo](https://callysto.ca/wp-content/uploads/2018/06/Callysto_Notebook-Banner_Top_06.06.18.jpg)](https://callysto.ca)

In [ ]:
import uiButtons
%uiButtons

# Ideas
- Sales tax examples
- using pie chart to visualize, maybe interactive
- Percent of students who own cats, dogs, both, or neither. Show as fractions contrasted with percentage. Maybe add inputs to get a class worth of data and include auto populate if not in a class setting (represent in pie chart?)
- Show how to go from fraction to percent using $\dfrac{x}{100}$ and $\dfrac{x}{y}$ with $y \neq 100$
- intro video or fun gif/image to engage (maybe win/loss ratio when playing a game)
- sports, goalie saves to shot ratio

https://www.kevinsidwar.com/iot/2017/7/1/the-undocumented-nhl-stats-api




In [ ]:
%%html
	<style>
		.mainBody {
			font-family: Arial, Helvetica, sans-serif;
		}
		#startBtn {
			background-color: cornflowerblue;
 			border: none;
 			border-radius: 3px;
  			font-size: 14px;
  			color: white;
  			font-weight: bold;
  			padding: 2px 8px;
  			text-transform: uppercase;
		}
	</style>
<div class="mainBody">
<div style="padding-bottom: 10px;">
		<label for="winningScore">Winning Score: </label>
		<select name="Winning Score" id="winningScore">
			<option value="3">3</option>
			<option value="5">5</option>
			<option value="7">7</option>
			<option value="10">10</option>
		</select>
		<button type="button" id="startBtn">Start</button>
	</div>
	<canvas id="gameCanvas" width="600" height="350" style="border: solid 1px black"></canvas>

	<div>
		<ul>
			<li>Player's point save average: <output id="playerAvg"></output></li>
			<li>Computer's point save average: <output id="compAvg"></output></li>
		</ul>
	</div>
</div>

## Question ideas
- once scored on once, can you ever get back to 1.0?

In [ ]:
%%html
<script>
		var canvas;
		var canvasContext;
		var isInitialized;

		var ballX = 50;
		var ballY = 50;
		var ballSpeedX = 5;
		var ballSpeedY = 3;

		var leftPaddleY = 250;
		var rightPaddleY = 250;

		var playerSaves = 0;
		var playerSOG = 0;
		var compSaves = 0;
		var compSOG = 0;

		var playerScore = 0;
		var compScore = 0;
		var winningScore = 3;
		var winScreen = false;

		const PADDLE_WIDTH = 10;
		const PADDLE_HEIGHT = 100;
		const BALL_RADIUS = 10;
		const COMP_SPEED = 4;

		document.getElementById('startBtn').onclick = function () {
			initGame();
			var selection = document.getElementById('winningScore');
			winningScore = Number(selection.options[selection.selectedIndex].value);
			canvas = document.getElementById('gameCanvas');
			canvasContext = canvas.getContext('2d');
			canvasContext.font = '50px Arial';
			ballReset();

			if (!isInitialized) {
				var framesPerSec = 60;
				setInterval(function () {
					moveAll();
					drawAll();
				}, 1000 / framesPerSec);
				isInitialized = true;
			}

			canvas.addEventListener('mousemove', function (event) {
				var mousePos = mouseYPos(event);
				leftPaddleY = mousePos.y - PADDLE_HEIGHT / 2;
			});
		}

		function updateSaveAvg() {
			var playerSaveAvgTxt = document.getElementById('playerAvg');
			var compSaveAvgTxt = document.getElementById('compAvg');

			var playerSaveAvg = playerSaves / playerSOG;
			var compSaveAvg = compSaves / compSOG;

			playerSaveAvgTxt.innerHTML = ((playerSaveAvg < 0 || isNaN(playerSaveAvg)) ? Number(0).toPrecision(3) + (' (0.0%)') :
				playerSaveAvg.toPrecision(3) + (' (' + (playerSaveAvg * 100).toPrecision(3) + '%)'));
			compSaveAvgTxt.innerHTML = ((compSaveAvg < 0 || isNaN(compSaveAvg)) ? Number(0).toPrecision(3) + (' (0.0%)') :
				compSaveAvg.toPrecision(
					3) + (' (' + (compSaveAvg * 100).toPrecision(3) + '%)'));

		}

		function initGame() {
			playerScore = 0;
			compScore = 0;
			playerSaves = 0;
			playerSOG = 0;
			compSaves = 0;
			compSOG = 0;
			ballSpeedX = 5;
			ballSpeedY = 3;
		}

		function ballReset() {
			if (playerScore >= winningScore || compScore >= winningScore) {
				winScreen = true;
			}
			if (winScreen) {
				updateSaveAvg();
				if (confirm('Another game?')) {
					winScreen = false;
                    initGame();
				} else {
					return;
				}
			}
			ballX = canvas.width / 2;
			ballY = canvas.height / 2;
			ballSpeedY = Math.floor(Math.random() * 4) + 1;
			var randomizer = Math.floor(Math.random() * 2) + 1;
			if (randomizer % 2 === 0) {
				ballSpeedY -= ballSpeedY;
			}
			flipSide();
		}

		function flipSide() {
			ballSpeedX = -ballSpeedX;
		}

		function moveAll() {
			if (winScreen) {
				return;
			}
			computerMove();
			ballX += ballSpeedX;
			if (ballX < (0 + BALL_RADIUS)) {
				if (ballY > leftPaddleY && ballY < leftPaddleY + PADDLE_HEIGHT) {
					playerSaves++;
					playerSOG++;
					flipSide();
					var deltaY = ballY - (leftPaddleY + PADDLE_HEIGHT / 2);
					ballSpeedY = deltaY * 0.35;
				} else {
					playerSOG++;
					compScore++;
					if (compScore === winningScore) {
						updateSaveAvg();
						drawAll();
						alert('Computer wins, final score: ' + playerScore + '-' + compScore);
					}
					ballReset();
				}
			}
			if (ballX >= canvas.width - BALL_RADIUS) {
				if (ballY > rightPaddleY && ballY < rightPaddleY + PADDLE_HEIGHT) {
					compSaves++;
					compSOG++;
					flipSide();
					var deltaY = ballY - (rightPaddleY + PADDLE_HEIGHT / 2);
					ballSpeedY = deltaY * 0.35;
				} else {
					compSOG++;
					playerScore++;
					if (playerScore === winningScore) {
						updateSaveAvg();
						drawAll();
						alert('You win, final score: ' + playerScore + '-' + compScore);
					}
					ballReset();
				}
			}
			ballY += ballSpeedY;
			if (ballY >= canvas.height - BALL_RADIUS || ballY < 0 + BALL_RADIUS) {
				ballSpeedY = -ballSpeedY;
			}
			updateSaveAvg();
		}

		function computerMove() {
			var rightPaddleYCenter = rightPaddleY + (PADDLE_HEIGHT / 2)
			if (rightPaddleYCenter < ballY - 20) {
				rightPaddleY += COMP_SPEED;
			} else if (rightPaddleYCenter > ballY + 20) {
				rightPaddleY -= COMP_SPEED;
			}
		}

		function mouseYPos(event) {
			var rect = canvas.getBoundingClientRect();
			var root = document.documentElement;
			var mouseX = event.clientX - rect.left - root.scrollLeft;
			var mouseY = event.clientY - rect.top - root.scrollTop;
			return {
				x: mouseX,
				y: mouseY
			};
		}

		function drawAll() {

			colorRect(0, 0, canvas.width, canvas.height, 'black');
			if (winScreen) {
				drawNet();
				drawScore();
				return;
			}
			//Left paddle
			colorRect(1, leftPaddleY, PADDLE_WIDTH, PADDLE_HEIGHT, 'white');
			//Right paddle
			colorRect(canvas.width - PADDLE_WIDTH - 1, rightPaddleY, PADDLE_WIDTH, PADDLE_HEIGHT, 'white');
			//Ball
			colorCircle(ballX, ballY, BALL_RADIUS, 'white');

			drawNet();

			drawScore();

		}

		function colorRect(x, y, width, height, drawColor) {
			canvasContext.fillStyle = drawColor;
			canvasContext.fillRect(x, y, width, height);
		}

		function colorCircle(centerX, centerY, radius, drawColor) {
			canvasContext.fillStyle = 'drawColor';
			canvasContext.beginPath();
			canvasContext.arc(centerX, centerY, radius, 0, Math.PI * 2, true);
			canvasContext.fill();
		}

		function drawScore() {
			canvasContext.fillText(playerScore, (canvas.width / 2) - (canvas.width / 4) - 25, 100);
			canvasContext.fillText(compScore, (canvas.width / 2) + (canvas.width / 4) - 25, 100);
		}

		function drawNet() {
			for (var i = 0; i < 60; i++) {
				if (i % 2 === 1) {
					colorRect(canvas.width / 2 - 3, i * 10, 6, 10, 'white')
				}
			}
		}
	</script>

[![logo](https://callysto.ca/wp-content/uploads/2018/07/Callysto-Notebook-Banner_Bottom_07.30.18.jpg)](https://callysto.ca/)